In [1]:
import json, pycm, pandas as pd
from nl_classifier import NaturalLanguageClassifer
from datetime import datetime

In [2]:
MODEL_NAME = 'gpt-4'
TEMPERATURE = 0.1
RESULTS_FILENAME = 'wd_gpt-4_serialization_from_wikidata_results.json'

In [3]:
experiments = json.load(open(RESULTS_FILENAME, 'r'))

In [4]:
for i, experiment in enumerate(experiments):
    cls = experiment["concept"]
    if "created" in experiment:
        print(f'{i+1:02}: {cls["label"]:30} {cls["id"]} SKIPPING')
    else:
        classifier = NaturalLanguageClassifer(cls["id"], cls["label"], cls["serialization"], MODEL_NAME, TEMPERATURE)
        print(f'{i+1:02}: {cls["label"]:30} {cls["id"]}')
        data = experiment["data"]
        for j, entity in enumerate(data):
            if "predicted" in entity:
                print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} SKIPPING')
            else:
                print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} {entity["actual"]:10} {classifier.tokens_used(entity["label"] + " " + entity["serialization"]):05} tokens ', end=" ")
                classification = classifier.classify(entity["label"], entity["serialization"])
                entity["predicted"] = classification["predicted"].lower()
                if entity["actual"] != entity["predicted"]:
                    if entity["actual"] == "positive":
                        print("FN")
                    else:
                        print("FP")
                else:
                    print("")
                entity["rationale"] = classification["rationale"]
            df_results = pd.DataFrame.from_records(data)
            cm = pycm.ConfusionMatrix(df_results["actual"].tolist(), df_results["predicted"].tolist(), digit=2, classes=[ 'positive', 'negative' ])
            experiment["confusion_matrix"] = cm.matrix
            experiment["created"] = datetime.now().isoformat()
            json.dump(experiments, open(RESULTS_FILENAME, 'w+'))


01: History of cities              http://www.wikidata.org/entity/Q30324006
   01: history of pest                http://www.wikidata.org/entity/Q266369        positive   00114 tokens  
   02: history of Hamburg             http://www.wikidata.org/entity/Q314618        positive   00110 tokens  
   03: history of Paris               http://www.wikidata.org/entity/Q845625        positive   00114 tokens  
   04: History of Hajdúdorog          http://www.wikidata.org/entity/Q1032658       positive   00098 tokens  
   05: history of New York City       http://www.wikidata.org/entity/Q1067131       positive   00126 tokens  
   06: history of Lyon                http://www.wikidata.org/entity/Q2029          positive   00110 tokens  
   07: history of Pirmasens           http://www.wikidata.org/entity/Q22818         positive   00129 tokens  
   08: history of Zürich              http://www.wikidata.org/entity/Q684180        positive   00116 tokens  
   09: history of Washington, D.C.    http:/